## GRU를 이용한 텍스트 생성
- 데이터: 캐글 New York Times Comments
- ArticlesApril2018.csv

In [1]:
import pandas as pd
from google.colab import files
up = files.upload()

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [2]:
df = pd.read_csv('ArticlesApril2018.csv', encoding='latin1')
df.head(3)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...


#### 데이터 전처리: headline column만 사용

In [3]:
df.headline.isna().sum()

0

In [4]:
headlines = df.headline.values.tolist()
headlines[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [5]:
# 노이즈 데이터('Unknown') 제거
headlines = [line for line in headlines if line != 'Unknown']
len(headlines)

1214

In [6]:
# encoding, 구둣점 제거, 소문자로 변환
import re
clean_lines = []
for line in headlines:
    line = line.encode('utf8').decode('ascii','ignore')
    line = re.sub('[^A-Za-z0-9 ]', '', line)
    clean_lines.append(line.lower())
clean_lines[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

#### Keras 텍스트 전처리

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [8]:
# 단어집합 생성
t = Tokenizer()
t.fit_on_texts(clean_lines)

In [9]:
# 단어집합 크기
vocab_size = len(t.word_index) + 1
vocab_size

3494

In [10]:
# 단어 생성
sequences = []
for line in clean_lines:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])
sequences[:5]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52]]

In [11]:
# sequence 중에서 제일 길이가 긴 것
max_len = max(len(seq) for seq in sequences)
max_len

24

In [12]:
sequences = pad_sequences(sequences, maxlen=max_len)
sequences[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          99,  269],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   99,
         269,  371],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   99,  269,
         371, 1115],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,   99,  269,  371,
        1115,  582],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,   99,  269,  371, 1115,
         582,   52]], dtype=int32)

In [13]:
X = sequences[:, :-1]
Y = to_categorical(sequences[:, -1])
X.shape, Y.shape

((7803, 23), (7803, 3494))

#### 모델 정의
- Embedding vector length: 10
- GRU unit의 갯수: 128

In [14]:
import numpy as np
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [16]:
model = Sequential([
    Embedding(vocab_size, 10, input_length=max_len-1),
    GRU(128),
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            34940     
                                                                 
 gru (GRU)                   (None, 128)               53760     
                                                                 
 dense (Dense)               (None, 3494)              450726    
                                                                 
Total params: 539426 (2.06 MB)
Trainable params: 539426 (2.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

%time hist= model.fit(X, Y, epochs=200, verbose=0)

CPU times: user 5min 16s, sys: 24.1 s, total: 5min 40s
Wall time: 6min 24s


In [18]:
hist.history['accuracy'][-1]

0.9143918752670288

- 모델 검증

In [19]:
up = files.upload()

Saving my_util.py to my_util.py


In [20]:
from my_util import sentence_generation

In [21]:
"""'former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'how a bag of texas dirt  became a times tradition'"""
print(sentence_generation(model, t, max_len, 'former', 8))
print(sentence_generation(model, t, max_len, 'epa', 8))
print(sentence_generation(model, t, max_len, 'how', 8))

former nfl cheerleaders settlement offer 1 and a meeting
epa chiefs ethics woes have echoes in his past
how to prevent a racist hoodie is mueller with
